## 1) Initiate ESMLPipelineFactory (Always run thic CELL below)
- To attach ESML controlplane to your project
- To point at `template-data` for the pipeline to know the schema of data.
    - NB! Azure machine learning pipelines need sample data. You need to have sample-data underneath the datalake folder structure:
    - `0` is recommended for `model_version folder`
        - Note: Ensure there is data under that version folder:  project002/11_diabetes_model_reg/inference/`0`/ds01_diabetes/in/dev/`1000/01/01/`
    - `1000-01-01 00:00:00.243860` is recommended for `date_folder`
        - Example: project002/11_diabetes_model_reg/inference/`1`/ds01_diabetes/in/dev/`1000/01/01/`
- To init the ESMLPipelinefactory

In [1]:
import sys
sys.path.insert(0, "../azure-enterprise-scale-ml/esml/common/")
from esml import ESMLProject
from baselayer_azure_ml_pipeline import ESMLPipelineFactory, esml_pipeline_types
 
p = ESMLProject() # Will search in ROOT for your copied SETTINGS folder '../settings/model/active/active_scoring_in_folder.json',
p.inference_mode = True
p.active_model = 11 # 10=titanic , 11=Diabetes
p.ws = p.get_workspace_from_config()
p_factory = ESMLPipelineFactory(p)

# Azure machine learling pipelines need sample data to know schema
# model_version= 0 meaning that ESML will find LATEST PROMOTED/best model, and not use a specific Model.versio to score with. It will read data from .../inference/0/... folder
model_version = 5 # 5 = DatabricksPipeline, 36=AutoMLStep, 16=PythonPipeline, 1-4 = AtomlRun (No pipeline)
p_factory.batch_pipeline_parameters[0].default_value = model_version 

training_datefolder = '1000-01-01 10:35:01.243860' # Will override active_scoring_in_folder.json'
p_factory.batch_pipeline_parameters[1].default_value = training_datefolder # overrides ESMLProject.date_scoring_folder.

Using lake_settings.json with ESML version 1.4 - Models array support including LABEL


## Azure ML Legacy model: "One time a day" - the below is needed to be done, to ensure Azure ML v1
- Set LEGACY mode - Azure ML v1 - since private link and DatabricksStep

In [2]:
print("NB! The below command you only need to run 1 time a day - then you can disable this cell. comment the code lines")
print("")
p.ws = p.get_workspace_from_config()
p.ws.update(v1_legacy_mode=True) # If you happen to have a workspace in v2 mode, and want to change back to v1 legacy mode

NB! The below command you only need to run 1 time a day - then you can disable this cell. comment the code lines



{'id': '/subscriptions/50ef5835-c45a-4c2e-a596-2a9e0e2a0a33/resourceGroups/dc-heroes-esml-project001-weu-dev-001-rg/providers/Microsoft.MachineLearningServices/workspaces/aml-prj001-weu-dev-001',
 'name': 'aml-prj001-weu-dev-001',
 'identity': {'principal_id': '32f3a889-570a-404b-bd49-29d7b761aa13',
  'tenant_id': '846f02b7-f92a-4053-9a99-094e5ba2e1a4',
  'type': 'SystemAssigned'},
 'location': 'westeurope',
 'type': 'Microsoft.MachineLearningServices/workspaces',
 'tags': {'Application Name': 'Enterprise Scale ML (ESML)',
  'BA ID': 'NA',
  'BCIO': 'Robin',
  'Business Area': 'NA',
  'Cost Center': '123456',
  'Resource Managed By': 'The Riddler',
  'TechnicalContact': 'batman@gothamcity.dc_',
  'Project': 'Batcave upgrade',
  'Description': 'ESML AI Factory'},
 'sku': 'Basic',
 'workspaceid': '14329cb6-9c78-4714-89e1-5bde1f975f93',
 'sdkTelemetryAppInsightsKey': '6d7b2b55-6d18-4126-a3ee-251f78cfa6e5',
 'description': 'Azure ML workspace for prj001 in ESML-dev environment. In AI Facto

# 2) `AUTO-GENERATE code: a snapshot folder`
<a id='2_generate_snapshot_folder'></a>

In [3]:
## Generate CODE - then edit it to get correct environments
p_factory.create_dataset_scripts_from_template(overwrite_if_exists=False) # Do this once, then edit them manually. overwrite_if_exists=False is DEFAULT

Did NOT overwrite script-files with template-files such as 'scoring_gold.py', since overwrite_if_exists=False


In [4]:
import sys
sys.path.insert(0, "../azure-enterprise-scale-ml/")
from esmlrt.interfaces.iESMLPipelineStepMap import IESMLPipelineStepMap
sys.path.insert(0, "../pipelines/M11/your_code/")
from ESMLPipelineStepMap import ESMLPipelineStepMap

map = ESMLPipelineStepMap() # TODO 4 YOU: You need to implement this class. See "your_code" folder
p_factory.use_advanced_compute_settings(map)

inf_map = map.get_inference_map(p.active_model['dataset_folder_names'])
inf_map[3] # prints  the map

Azure ML Workspace:
Attached Databricks db_compute_name:
Compute target n1-p000-aml-91 already exists


{'step_name': 'scoring_gold',
 'code': '/Repos/jostrm@microsoft.com/esml-aifactory002-prj002/notebook_databricks/esml/dev/project/11_diabetes_model_reg/M11/30_scoring_gold',
 'compute_type': 'dbx',
 'date_folder_or': None,
 'dataset_folder_names': '',
 'dataset_filename_ending': '*.parquet',
 'compute_name': 'n1-p000-aml-91',
 'cluster_id': '0111-230838-10wcl6d4'}

# 3) `BUILDS the pipeline, and RUN the pipeline (smoke testing)`

Take note on the `esml_pipeline_types` below, of type: esml_pipeline_types.`IN_2_GOLD_SCORING`

In [7]:
## BUILD
batch_pipeline = p_factory.create_batch_pipeline(esml_pipeline_types.IN_2_GOLD_SCORING_DBX) # Note the esml_pipeline_types

Using GEN2 as Datastore
use_project_sp_2_mount: True
Environment ESML-AzureML-144-AutoML_131 exists
Using Azure ML Environment: 'ESML-AzureML-144-AutoML_131' as primary environment for PythonScript Steps
Dataset: ds01_diabetes has advanced mapping - an Azure Databricks mapping
Dataset: ds02_other has advanced mapping - an Azure Databricks mapping
ESML advanced mode: with advanced compute mappings
 - Step: silver_merged_2_gold has advanced mapping - an Azure Databricks mapping
Found attached Databricks compute cluster
previous_step_is_databricks = 1
create_gold_to_score_step: inference_mode=True
par_date_utc: 1000-01-01 10:35:01.243860
Created Databricks step in pipeline
Adding inference step, creating...
 - Step: scoring_gold has advanced mapping - an Azure Databricks mapping for: IN_2_GOLD_SCORING_DBX
previous_step_is_databricks = 1
ESML INFO: SPECIFIC (maybe leading) MODEL: model_version in-parameter from user is 5, hence no guarantee that BEST LATEST PROMOTED model.version is used. 

# 4a) `Execute the pipeline (smoke testing)`

In [ ]:
## RUN for smoke testing purpose, to see that it works during runtime
pipeline_run = p_factory.execute_pipeline(batch_pipeline) # Tip: Pointing at the wrong folder for the sample data is the most common error "StreamAccessException"
pipeline_run.wait_for_completion(show_output=False)

Execute_pipeline (scoring): Inference_mode: 1
-Scoring data, default value 1000-01-01 10:35:01.243860
Created step in2silver_ds01_diabetes [cc5c81f5][73875d3b-07f2-44ab-80b9-b9535f6f010d], (This step will run and generate new outputs)Created step in2silver_ds02_other [f5de5ff5][a583c9b8-1d32-4e38-bc7b-0622b386439a], (This step will run and generate new outputs)

Created step silver_merged_2_gold [8e1ea736][19b449ea-0077-45de-8205-a41d7b86a4fe], (This step will run and generate new outputs)
Created step scoring_gold [ee463133][6897be41-39ca-42d3-90fd-635499332715], (This step will run and generate new outputs)
Created data reference M11_ds01_diabetes_inference_IN for StepId [ce91d52c][260a2074-efd3-41f9-8336-1554c424c306], (Consumers of this data will generate new runs.)
Created data reference M11_ds02_other_inference_IN for StepId [7a50720c][abc46210-73cb-49ae-9419-793937440b22], (Consumers of this data will generate new runs.)
Submitted PipelineRun e512d93a-ace7-4c91-a944-91eae6deb265

ActivityFailedException: ActivityFailedException:
	Message: Activity Failed:
{
    "error": {
        "code": "UserError",
        "message": "Pipeline has failed child jobs. For more details and logs, please go to the job detail page and check the child jobs.",
        "messageFormat": "Pipeline has failed child jobs. {0}",
        "messageParameters": {},
        "referenceCode": "PipelineHasStepJobFailed",
        "details": []
    },
    "environment": "westeurope",
    "location": "westeurope",
    "time": "2023-04-25T21:33:15.195943Z",
    "componentName": ""
}
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Activity Failed:\n{\n    \"error\": {\n        \"code\": \"UserError\",\n        \"message\": \"Pipeline has failed child jobs. For more details and logs, please go to the job detail page and check the child jobs.\",\n        \"messageFormat\": \"Pipeline has failed child jobs. {0}\",\n        \"messageParameters\": {},\n        \"referenceCode\": \"

# 4b) See the RESULTS: Metadata about SCORING & actual SCORING

In [ ]:
from azureml.core import Dataset
from azureml.data import FileDataset
import pandas as pd

ds_name ="{}_GOLD_SCORED_RUNINFO".format(p.ModelAlias)
meta_ds= Dataset.get_by_name(workspace=p.ws,name=ds_name, version='latest')
pd.set_option('display.max_colwidth', None)

try:
    meta_ds.to_pandas_dataframe().head()
except AttributeError as e:
    ds_name ="{}_GOLD_SCORED".format(p.ModelAlias)
    print("ds_name", ds_name)
    meta_ds= Dataset.get_by_name(workspace=p.ws,name=ds_name, version='latest')

    if(type(meta_ds) is FileDataset):
        print("FileDataset = True")
        path = meta_ds.take(1).download('./data_temp/', overwrite=True)
        #df = pd.DataFrame(meta_ds.to_path())
        df = pd.DataFrame(path)
        df.head()
    else:
        print("TabularDataset = True")
        print(meta_ds.to_pandas_dataframe().head())
    


ds_name M11_GOLD_SCORED
FileDataset = True


UserErrorException: UserErrorException:
	Message: Execution failed with error message: ScriptExecutionException was caused by StreamAccessException.
  StreamAccessException was caused by NotFoundException.
    'AdlsGen2-ListFiles (req=1, existingItems=0)' for '[REDACTED]' on storage failed with status code 'NotFound' (The specified path does not exist.), client request ID '210e8fbf-0f0b-42aa-a686-22c4e92a5294', request ID '317205f2-101f-00a6-08c7-73e54e000000'. Error message: [REDACTED]
| session_id=699c1247-0705-4170-ab49-0f252979f0b6 ErrorCode: ScriptExecution.StreamAccess.NotFound
	InnerException 
Error Code: ScriptExecution.StreamAccess.NotFound
Failed Step: 853ea58d-a95b-4e7b-b3eb-1e523964d20a
Error Message: ScriptExecutionException was caused by StreamAccessException.
  StreamAccessException was caused by NotFoundException.
    'AdlsGen2-ListFiles (req=1, existingItems=0)' for 'https://dcheroesgderqesml001dev.dfs.core.windows.net/lake3?directory=azureml/9e7f193a-bd29-4097-ae

# 5) PUBLISH the TRAINING pipeline & PRINT its ID

In [7]:
# PUBLISH
published_pipeline, endpoint = p_factory.publish_pipeline(batch_pipeline,"_1") # "_1" is optional    to create a NEW pipeline with 0 history, not ADD version to existing pipe & endpoint

# PRINT: Get info to use in Azure data factory
- `published_pipeline.id` (if private Azure ML workspace)

In [8]:
print("2) Fetch scored data: Below needed for Azure Data factory PIPELINE activity (Pipeline OR Endpoint. Choose the latter") 
print ("- Endpoint ID")
print("Endpoint ID:  {}".format(endpoint.id))
print("Endpoint Name:  {}".format(endpoint.name))
print("Experiment name:  {}".format(p_factory.experiment_name))

print("In AZURE DATA FACTORY - This is the ID you need, if using PRIVATE LINK, private Azure ML workspace.")
print("-You need PIPELINE id, not pipeline ENDPOINT ID ( since cannot be chosen in Azure data factory if private Azure ML)")
published_pipeline.id

2) Fetch scored data: Below needed for Azure Data factory PIPELINE activity (Pipeline OR Endpoint. Choose the latter
- Endpoint ID
Endpoint ID:  aedd1e4c-a3eb-40ae-b9c2-a93b6b576523
Endpoint Name:  11_diabetes_model_reg_pipe_IN_2_GOLD_SCORING_EP_1
Experiment name:  11_diabetes_model_reg_pipe_IN_2_GOLD_SCORING
In AZURE DATA FACTORY - This is the ID you need, if using PRIVATE LINK, private Azure ML workspace.
-You need PIPELINE id, not pipeline ENDPOINT ID ( since cannot be chosen in Azure data factory if private Azure ML)


'1930f46b-69ab-4aec-9e93-f6d5998c9e7c'

 # DONE! Next step would be

 - Q: `Next step in PRODUCTION phaase after the 2a and 3a or 3b notebooks are done?`

1) Go to your ESMLProjects `Azure data factory`, and use the `ESML DataOps templates` (Azure data factory templates) for `IN_2_GOLD_SCORING`
    - azure-enterprise-scale-ml\copy_my_subfolders_to_my_grandparent\adf\v1_3\PROJECT000\LakeOnly\`STEP03_IN_2_GOLD_SCORING.zip`
2) Go to the next notebook `mlops` folder, to setup `CI/CD` in Azure Devops
    - Import this in Azure devops
        azure-enterprise-scale-ml\copy_my_subfolders_to_my_grandparent\mlops\01_template_v14\azure-devops-build-pipeline-to-import\\`ESML-v14-project002_M11-DevTest.json`
    - Change the Azure Devops `VARIABLES` for service principle, tenant, etc.
    - Change parameters in the `inlince Azure CLI script` to correct model you want to work with, and the correct data you want to train with, or score.
        - File: `31-deploy_and_smoketest_batch_scoring.py`
        - INLINE code: `--esml_model_number 11 --esml_date_utc "1000-01-01 10:35:01.243860"`